In [11]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py


from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense,GlobalAveragePooling2D,BatchNormalization,Dropout
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import sklearn.metrics 
from sklearn.metrics import classification_report, confusion_matrix

trainingSet = 'trainSet'
testSet = 'testSet'

As before, we need to uniform all the images to the same size and resolutions:

In [9]:

batch_size = 32
trainDatagen = ImageDataGenerator(
    rescale = 1. / 255,\
    zoom_range=0.1,\
    rotation_range=10,\
    width_shift_range=0.1,\
    height_shift_range=0.1,\
    horizontal_flip=True,\
    vertical_flip=False)

trainGenerator = trainDatagen.flow_from_directory(
    directory=trainingSet,
    target_size=(256,256),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True
)

testDatagen = ImageDataGenerator(
    rescale = 1. / 255)

testGenerator = testDatagen.flow_from_directory(
    directory=testSet,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False
)

num_samples = trainGenerator.n
num_classes = trainGenerator.num_classes
input_shape = trainGenerator.image_shape

classnames = [k for k,v in trainGenerator.class_indices.items()]

print("Image input %s" %str(input_shape))
print("Classes: %r" %classnames)

print('Loaded %d training samples from %d classes.' %(num_samples,num_classes))
print('Loaded %d test samples from %d classes.' %(testGenerator.n,testGenerator.num_classes))


Found 2000 images belonging to 4 classes.
Found 3038 images belonging to 4 classes.
Image input (256, 256, 3)
Classes: ['HAZE', 'RAINY', 'SNOWY', 'SUNNY']
Loaded 2000 training samples from 4 classes.
Loaded 3038 test samples from 4 classes.


Now we can add new layers to our trained model:

In [ ]:
x=Dense(1024,activation='relu')(x) #we add dense layers so that the model can learn more complex functions and classify for better results.
x=BatchNormalization()(x)
x=Dropout(0.5)(x)
x=Dense(1024,activation='relu')(x) #dense layer 2
x=BatchNormalization()(x)
x=Dropout(0.25)(x)
x=Dense(512,activation='relu')(x) #dense layer 3
x=BatchNormalization()(x)
x=Dropout(0.25)(x)
preds=Dense(4,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)


for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True

model.summary()

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

steps_per_epoch=trainGenerator.n//trainGenerator.batch_size
val_steps=testGenerator.n//testGenerator.batch_size+1

print(steps_per_epoch)
print(val_steps)


checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_accuracy', verbose=1, save_best_only=True)

history = model.fit_generator(trainGenerator, epochs=20,callbacks=[checkpoint], verbose=1,\
                    steps_per_epoch=steps_per_epoch,\
                    validation_data=testGenerator,\
                    validation_steps=val_steps)

model.load_weights(checkpath)

Now we can plot the confusion matrix:

In [ ]:

preds = model.predict_generator(testGenerator,verbose=1,steps=val_steps)

Ypred = np.argmax(preds, axis=1)
Ytest = testGenerator.classes  # shuffle=False in test_generator

print(classification_report(Ytest, Ypred, labels=None, target_names=classnames, digits=3))


cm = confusion_matrix(Ytest, Ypred)

conf = [] # data structure for confusions: list of (i,j,cm[i][j])
for i in range(0,cm.shape[0]):
  for j in range(0,cm.shape[1]):
    if (i!=j and cm[i][j]>0):
      conf.append([i,j,cm[i][j]])

col=2
conf = np.array(conf)
conf = conf[np.argsort(-conf[:,col])]  # decreasing order by 3-rd column (i.e., cm[i][j])

print('%-16s     %-16s  \t%s \t%s ' %('True','Predicted','errors','err %'))
print('------------------------------------------------------------------')
for k in conf:
  print('%-16s ->  %-16s  \t%d \t%.2f %% ' %(classnames[k[0]],classnames[k[1]],k[2],k[2]*100.0/test_generator.n))
  

Now we can plot our results:

In [ ]:

for i in history.history :
  print(i)
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()